In [14]:
import pandas as pd

https://gengo.ai/datasets/15-free-sentiment-analysis-datasets-for-machine-learning/

In [15]:
import json
i = 0
reviews_lst_good= []
reviews_lst_bad = []


with open('Books_5.json', 'r') as f:
        for line in f:
            if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<=500000:
                reviews_lst_good.append (json.loads(line))
            
            elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<=500000:
                reviews_lst_bad.append (json.loads(line))

            if len(reviews_lst_good) + len (reviews_lst_bad) == 1000000:
                break

In [16]:
reviewerID_lst = []
reviewerName_lst = []
reviewText_lst = []
overall_lst = []
summary_lst = []

# reviews_lst [0]['overall']<5 #['asin']for 

for i in reviews_lst_good:
    reviewerID_lst.append (i['reviewerID'])
    reviewText_lst.append (i['reviewText'])
    overall_lst.append (i['overall'])
    summary_lst.append (i['summary'])
    
for i in reviews_lst_bad:
    reviewerID_lst.append (i['reviewerID'])
    reviewText_lst.append (i['reviewText'])
    overall_lst.append (i['overall'])
    summary_lst.append (i['summary'])
    
df = pd.DataFrame ()

df['reviewerID'] = reviewerID_lst
# df['reviewerName'] = reviewerName_lst
df['reviewText'] = reviewText_lst
df['overall'] = overall_lst
df['summary'] = summary_lst

In [32]:
df ['mark'] = 0
df.loc[df['overall'] > 3,'mark'] = 1

print (len(df[df['mark'] > 0]))

500001


In [33]:
df[(df['mark'] != 0)&(df['mark'] != 1)]

,reviewerID,reviewText,overall,summary,mark


In [34]:
reviews_train = list(df[df['overall'] > 3]['reviewText'][:-10000]) + list(df[df['overall'] < 3]['reviewText'][:-10000])
reviews_test = list(df[df['overall'] > 3]['reviewText'][-10000:]) + list(df[df['overall'] < 3]['reviewText'][-10000:])

In [35]:
# list(df[df['overall'] > 3]['reviewText'][:2])
reviews_train [:2]

['Spiritually and mentally inspiring! A book that allows you to question your morals and will help you discover who you really are!',
 "This is one my must have books. It is a masterpiece of spirituality. I'll be the first to admit, its literary quality isn't much. It is rather simplistically written, but the message behind it is so powerful that you have to read it. It will take you to enlightenment."]

In [36]:
import re
from nltk.corpus import stopwords 

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)


In [37]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [38]:

y = list(df[df['overall'] > 3]['mark'][:-10000]) + list(df[df['overall'] < 3]['mark'][:-10000])

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)

# for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
#     lr = LogisticRegression(C=c)
#     lr.fit(X_train, y_train)
#     print ("Accuracy for C=%s: %s" 
#            % (c, accuracy_score(y_val, lr.predict(X_val))))

In [39]:
final_model = LogisticRegression(C=0.5)
final_model.fit(X,  y)
print ("Final Accuracy: %s" 
       % accuracy_score( list(df[df['overall'] > 3]['mark'][-10000:]) + list(df[df['overall'] < 3]['mark'][-10000:])
                        , final_model.predict(X_test)))

Final Accuracy: 0.8996


In [40]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)

('klausner', 4.217558096589567)
('ariely', 2.848116519598753)
('everneath', 2.7092255946127577)
('spong', 2.5704121614981528)
('maxon', 2.5124985059202687)
('booksneeze', -3.493254457182336)
('waterbrook', -2.7632559624305277)
('killgore', -2.69149943606483)
('scarpetta', -2.6093523972060035)
('booksneezecom', -2.5758875198341338)


In [41]:
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:120]:
    print (best_positive)

writers = ['klausner','rollins','ehrman','gaiman','spong','olaf','steinbeck','harpercollins','discworld',
'kingsolver','pratchett','maxon']

('klausner', 4.217558096589567)
('ariely', 2.848116519598753)
('everneath', 2.7092255946127577)
('spong', 2.5704121614981528)
('maxon', 2.5124985059202687)
('olaf', 2.3654402805132566)
('ehrman', 2.358455133342693)
('aria', 2.2955011982448776)
('pleasantly', 2.2493119170369544)
('margolin', 2.0781589783203493)
('erdrich', 2.045893274820339)
('hobb', 1.9770975521332592)
('proses', 1.9715490922912755)
('tamani', 1.9712774311695884)
('maclean', 1.9689074781163967)
('kaiden', 1.968689765074976)
('humbert', 1.9662899890904848)
('zinsser', 1.9507065017084348)
('wynne', 1.9408599770903854)
('gaiman', 1.9391932700856813)
('wallis', 1.9284076717647278)
('lobdell', 1.917548648755522)
('quibble', 1.9137553831651555)
('rollins', 1.9127985800650966)
('refreshing', 1.9093870931130132)
('harpercollins', 1.9052766237864243)
('argeneau', 1.903691571827448)
('halpern', 1.8960486023895722)
('toole', 1.8875415002401268)
('septimus', 1.876508140172002)
('refreshingly', 1.8698882301786124)
('renni', 1.86571

In [ ]:
# BackUP

In [ ]:
# df = pd.read_json('Books_5.json')
# import ijson
# filename = "Books_5.json"
# with open(filename, 'r') as f:
#     objects = ijson.items(f, 'meta.view.columns.item')
#     columns = list(objects)
#     print (objects)
# handle = open("Books_5.json", "r")

# while True:
#     data = handle.read(20000)
#     if not data:
#         break